<a href="https://colab.research.google.com/github/oumaymadahmani/ML/blob/main/TP1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q kaggle

In [7]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oumaymadahmani","key":"e1551cd869697d0387d651d55c33e536"}'}

In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 96% 162M/169M [00:01<00:00, 100MB/s]
100% 169M/169M [00:01<00:00, 110MB/s]


In [11]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
df=pd.read_csv('/content/sample.csv')

In [ ]:
df

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0
...,...,...,...,...,...,...,...
88,119330,105859,True,Wed Oct 11 13:50:42 +0000 2017,@105860 I wish Amazon had an option of where I...,119329,119331.0
89,119331,105860,True,Wed Oct 11 13:47:14 +0000 2017,They reschedule my shit for tomorrow https://t...,119330,NaN
90,119332,Tesco,False,Wed Oct 11 13:34:06 +0000 2017,"@105861 Hey Sara, sorry to hear of the issues ...",119333,119334.0
91,119333,105861,True,Wed Oct 11 14:05:18 +0000 2017,@Tesco bit of both - finding the layout cumber...,"119335,119336",119332.0


In [14]:
l=df["text"]
l

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text, Length: 93, dtype: object

In [17]:
import string
import nltk
from nltk.stem.porter import PorterStemmer
import spacy
import re
from collections import Counter

nltk.download('stopwords')

# Assuming df is a pandas DataFrame and "text" is one of its columns
df["text_cleaned"] = df["text"].str.lower()

# Removal of Punctuations
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """Custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Apply the remove_punctuation function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_punctuation)

# Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """Custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Apply the remove_stopwords function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_stopwords)

# Removal of Frequent words
cnt = Counter()
for text_entry in df["text_cleaned"].values:
    for word in text_entry.split():
        cnt[word] += 1

# Get the 10 most common words
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text_entry):
    """Custom function to remove the frequent words"""
    return " ".join([word for word in str(text_entry).split() if word not in FREQWORDS])

# Apply the remove_freqwords function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_freqwords)

# Removal of Rare words
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text_entry):
    """Custom function to remove the rare words"""
    return " ".join([word for word in str(text_entry).split() if word not in RAREWORDS])

# Apply the remove_rarewords function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_rarewords)

# Stemming
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

# Apply the stem_words function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(stem_words)

# Lemmatization
nlp = spacy.load("en_core_web_sm")
def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Apply the lemmatize_words function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(lemmatize_words)

# Removal of Emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Apply the remove_emoji function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_emoji)

# Removal of Emoticons
EMOTICONS = {
    # Define your emoticons here
}

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

# Apply the remove_emoticons function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_emoticons)

# Removal of URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

# Apply the remove_urls function to the "text_cleaned" column
df["text_cleaned"] = df["text_cleaned"].apply(remove_urls)




# Affichez le DataFrame avec les trois colonnes
print(df[['tweet_id', 'text', 'text_cleaned']])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


    tweet_id                                               text  \
0     119237  @AppleSupport causing the reply to be disregar...   
1     119238  @105835 Your business means a lot to us. Pleas...   
2     119239  @76328 I really hope you all change but I'm su...   
3     119240  @105836 LiveChat is online at the moment - htt...   
4     119241  @VirginTrains see attached error message. I've...   
..       ...                                                ...   
88    119330  @105860 I wish Amazon had an option of where I...   
89    119331  They reschedule my shit for tomorrow https://t...   
90    119332  @105861 Hey Sara, sorry to hear of the issues ...   
91    119333  @Tesco bit of both - finding the layout cumber...   
92    119335  @105861 If that doesn't help please DM your fu...   

                                         text_cleaned  
0     caus repli disregard tap notif keyboard open     
1   105835 busi mean lot name zip code addit detai...  
2    76328 realli hope chan